<hr>

## ) Ivar Walskaar

<hr>

<br>

# Loading Datasets

In [3]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import spacy

# Load Lemmatizer
nlp = spacy.load("en_core_web_sm")

# Load Stopwords
nltk.download('stopwords')
cachedStopWords = list(set(stopwords.words("english")))

# Load Twitter Datasets - subtask a)
df_train = pd.read_csv("data/1a/CT22_english_1A_checkworthy_train.tsv", sep='\t')
df_val = pd.read_csv("data/1a/CT22_english_1A_checkworthy_dev.tsv", sep='\t')
df_test = pd.read_csv("data/1a/CT22_english_1A_checkworthy_dev_test.tsv", sep='\t')

[nltk_data] Downloading package stopwords to /Users/ivarw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<br>

### Class Priors

In [4]:
samples = len(df_train)
class1 = (df_train["class_label"] == 0).sum()
class2 = (df_train["class_label"] == 1).sum()
print(f"Class 1 (Not Worthy): {round(class1/samples, 2)}")
print(f"Class 2 (Worthy): {round(class2/samples, 2)}")

Class 1 (Not Worthy): 0.79
Class 2 (Worthy): 0.21


<br><br><br>

# 1) Preprocessing

<br>

### Balancing of Datasets (3 Approaches)

In [5]:
# 0:unworthy = 1675, 1:worthy = 447
sample_num = df_train["class_label"].value_counts() 

# Separate true and false
df_false = df_train[df_train["class_label"] == 0]
df_true = df_train[df_train["class_label"] == 1]

# Downsample majority class
df_train_downsampled = pd.concat([df_false.sample(sample_num[1]), df_true])

# Upsample minority class with replace=true
df_train_upsampled = pd.concat([df_false, df_true.sample(sample_num[0], replace=True)])

# Hybrid-sampling
midpoint = int((sample_num[0] - sample_num[1]) / 2)
df_train_hybrid = pd.concat([df_false.sample(midpoint), df_true.sample(midpoint, replace=True)])

<br>

### Emoji Removal (used in custom tokenizer)

In [6]:
# Function to remove emojies when using the lemmatizer
#https://poopcode.com/how-to-remove-emoji-from-text-in-python/
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

<br><br><br>

### Custom Tokenizers

In [7]:
def tokenize_text(text, method="stopwords"):
    
    # Remove hyperlinks
    text = re.sub(r'http\S+', '', text)
    
    # Split sentence into words
    tokens = []
    for token in ngrams(text.split(), 1):
        word = re.sub(r',', '', token[0]) #Remove commas
        word = re.sub(r'[\!\.\:]$', '', word) #Remove (.!:) at the end of tokens
        word = re.sub(r'#', '', word) #Remove hashtags (not the text)
        if word == "—": continue #Ignore dash
        if word.find("@") != -1: continue #Ignore tags
        tokens.append(word)
    tokens

    # List comprehension to remove stopwords
    if method != "default":
        tokens = [x for x in tokens if x not in cachedStopWords]

    # Perform stemming
    if method == "stemming":
        ps = PorterStemmer()
        tokens = [ps.stem(x) for x in tokens]

    # Perform lemmatization
    if method == "lemmatizer":
        # Remove emojies due to lemmatizer not handling them well
        tokens = [x for x in tokens if remove_emoji(x) == x]
        text = " ".join(tokens)
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc]

    return tokens

In [8]:
def tokenize_normal(text):
    return tokenize_text(text, method="default")

def tokenize_stopwords(text):
    return tokenize_text(text, method="stopwords")

def tokenize_stemming(text):
    return tokenize_text(text, method="stemming")

def tokenize_lemmatizer(text):
    return tokenize_text(text, method="lemmatizer")

In [9]:
def tokenize_df(df, text_col, method="default"):
    article_tokens = []
    for i in range(len(df)):
        text = df.iloc[i][text_col].lower()
        article_tokens.append(tokenize_text(text, method))
    df["tokens"] = article_tokens

In [10]:
# Tokenize training datasets
tokenize_df(df_train, "tweet_text", "stemming")
tokenize_df(df_train_downsampled, "tweet_text", "stemming")
tokenize_df(df_train_upsampled, "tweet_text", "stemming")
tokenize_df(df_train_hybrid, "tweet_text", "stemming")

# Tokenize test and validation data
tokenize_df(df_val, "tweet_text", "stemming")
tokenize_df(df_test, "tweet_text", "stemming")

In [11]:
print(df_train["tokens"][0], "\n")
print(df_val["tokens"][0], "\n")
print(df_test["tokens"][0])

["india'", 'gift', '100000', 'covid-19', 'vaccin', 'arriv', 'barbado', 'earlier', 'today', 'special', 'moment', 'barbadian', 'want', 'thank', 'prime', 'minist', 'modi', 'quick', 'decis', 'magnanim', 'action', 'allow', 'us', 'beneficiari', 'vaccin'] 

['nativ', 'american', 'commun', 'hit', 'hard', 'covid-19', 'thank', 'indian', 'health', 'servic', 'strong', 'partnership', 'tribal', 'govern', 'organ', 'urban', 'indian', 'group', '500000', 'vaccin', 'alreadi', 'administ', 'way'] 

['senat', 'pass', 'covid', 'relief', '✔️', '$1400', 'relief', 'check', '✔️', 'fund', 'vaccin', '✔️', 'money', 'reopen', 'school', '✔️', 'food', 'unemploy', 'rental', 'assist', '✔️', 'cut', 'child', 'poverti', 'half', '✔️', 'help', 'small', 'busi', 'must', 'end', 'pandem', 'help', 'way']


<br><br><br>

### Create Ngrams

In [12]:
def create_ngrams(df, n=1):
    # Don't execute code if no tokens column
    if "tokens" not in df.columns.tolist():
        print("The dataframe does not have a tokens columns")
        return

    # If monogram, copy token column to ngram column
    if n == 1:
        df["ngrams"] = df["tokens"]

    # Create ngrams
    article_ngrams = []
    for i in range(len(df)):
        ngrams = []
        tokens = df.iloc[i]["tokens"]
        for j in range(len(tokens)-n-1):
            ngrams.append(' '.join(tokens[j:j+n]))
        article_ngrams.append(ngrams)
    df["ngrams"] = article_ngrams
        

In [13]:
create_ngrams(df_train, n=1)
create_ngrams(df_val, n=1)
create_ngrams(df_test, n=1)

In [14]:
print(df_train.iloc[0]["ngrams"], "\n")
print(df_val.iloc[0]["ngrams"], "\n")
print(df_test.iloc[0]["ngrams"])

["india'", 'gift', '100000', 'covid-19', 'vaccin', 'arriv', 'barbado', 'earlier', 'today', 'special', 'moment', 'barbadian', 'want', 'thank', 'prime', 'minist', 'modi', 'quick', 'decis', 'magnanim', 'action', 'allow', 'us'] 

['nativ', 'american', 'commun', 'hit', 'hard', 'covid-19', 'thank', 'indian', 'health', 'servic', 'strong', 'partnership', 'tribal', 'govern', 'organ', 'urban', 'indian', 'group', '500000', 'vaccin', 'alreadi'] 

['senat', 'pass', 'covid', 'relief', '✔️', '$1400', 'relief', 'check', '✔️', 'fund', 'vaccin', '✔️', 'money', 'reopen', 'school', '✔️', 'food', 'unemploy', 'rental', 'assist', '✔️', 'cut', 'child', 'poverti', 'half', '✔️', 'help', 'small', 'busi', 'must', 'end', 'pandem']


<br><br><br>

<hr>

# Feature Exploration & Visualization

In [15]:
from sklearn.feature_extraction.text import CountVectorizer # For creating a DTM (discrete values)
from sklearn.feature_extraction.text import TfidfVectorizer # A weighted DTM (fractions)
#from wordcloud import WordCloud

<br>

### Example: Unfiltered Vocabulary 

In [16]:
# With stopwords
cv1 = CountVectorizer(tokenizer=tokenize_normal, ngram_range=(1, 1), max_features=5000)
tweet_vec = cv1.fit(df_train["tweet_text"]) # DTM (CV, but normalized for relative frequency)
tweet_bow_train = tweet_vec.transform(df_train["tweet_text"])

captured = tweet_bow_train.sum()
print(f"Total words across all tweets: 71346")
print(f"Total words captured by our features/vocabulary {captured} ({round(100*captured/71346, 1)}%)")
print(f"Amount of unique words (tokens/features): {len(tweet_vec.vocabulary_)}\n") # Unique words (features)

sum_words = tweet_bow_train.sum(axis=0) # Sum the columns from the DTM matrix
words_freq = [(word, sum_words[0, index]) for word, index in tweet_vec.vocabulary_.items()] # Make a list pairing every word with its corresponding DTM column summation
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[0:30])

#WordCloud.generate_from_frequencies(words_freq)

/Users/ivarw/miniforge3/envs/DAT540/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:514: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Total words across all tweets: 71346
Total words captured by our features/vocabulary 65975 (92.5%)
Amount of unique words (tokens/features): 5000

[('the', 3279), ('and', 2457), ('to', 2162), ('vaccines', 1730), ('of', 1633), ('covid-19', 1276), ('in', 1175), ('a', 1049), ('for', 950), ('vaccine', 892), ('is', 728), ('are', 662), ('on', 514), ('that', 482), ('have', 432), ('with', 410), ('at', 380), ('this', 373), ('will', 351), ('from', 341), ('by', 336), ('covid', 322), ('be', 321), ('more', 318), ('has', 315), ('it', 306), ('we', 294), ('as', 283), ('all', 273), ('you', 263)]


<br>

### Example: Stop-Words-Filtered Vocabulary

In [17]:
# Without stopwords
cv2 = CountVectorizer(tokenizer=tokenize_stopwords, ngram_range=(1, 1), max_features=5000)
tweet_vec = cv2.fit(df_train["tweet_text"]) # DTM (CV, but normalized for relative frequency)
tweet_bow_train = tweet_vec.transform(df_train["tweet_text"]) 

sum_words = tweet_bow_train.sum(axis=0) # Sum the columns from the DTM matrix
words_freq = [(word, sum_words[0, index]) for word, index in tweet_vec.vocabulary_.items()] # Make a list pairing every word with its corresponding DTM column summation
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[0:30])

#WordCloud.generate_from_frequencies(words_freq)

[('vaccines', 1730), ('covid-19', 1276), ('vaccine', 892), ('covid', 322), ('people', 259), ('coronavirus', 234), ('get', 232), ('covid19', 227), ('health', 211), ('&amp;', 198), ('first', 191), ('us', 167), ('doses', 165), ('new', 163), ('countries', 142), ('one', 132), ('effective', 128), ('today', 124), ('-', 123), ('safe', 120), ('available', 115), ('pandemic', 113), ('world', 111), ('march', 110), ('india', 105), ('cases', 105), ('vaccination', 104), ('virus', 101), ('said', 100), ('pfizer', 99)]


<br><br><br>

<hr>

# ) Optimization

<br><br>

## Load Report

In [18]:
df_results = {}

### Adjust Parameters

In [19]:
cv = CountVectorizer() 
tfidf = TfidfVectorizer()

VECTORIZERS = [cv, tfidf]
#TRAIN_SET = [df_train, df_train_downsampled, df_train_upsampled, df_train_hybrid]
TOKENIZERS = [tokenize_normal, tokenize_stopwords, tokenize_stemming, tokenize_lemmatizer]
NGRAM_SIZE = [(1,1), (2,2), (3,3)]
VOCAB_SIZE = [500, 2500, 5000, 50000]

In [20]:
#train_set = [df_train, df_train_hybrid, df_train_upsampled, df_train_downsampled]
train_set = df_train

vec = VECTORIZERS[0]
vec.tokenizer = TOKENIZERS[1]
vec.ngram_range = NGRAM_SIZE[0]
vec.max_features = VOCAB_SIZE[2]

tweet_vec = vec.fit(train_set["tweet_text"]) # DTM (CV, but normalized for relative frequency)
tweet_bow_train = tweet_vec.transform(train_set["tweet_text"]) 
tweet_bow_val = tweet_vec.transform(df_val["tweet_text"]) 
tweet_bow_test = tweet_vec.transform(df_test["tweet_text"]) 

<br><br><br>

## Baseline Predictions

In [21]:
from sklearn.dummy import DummyClassifier
from sklearn import metrics

dummy_clf = DummyClassifier(strategy="prior")
dummy_clf.fit(tweet_bow_train, train_set["class_label"])
print(f"Priors: {dummy_clf.class_prior_}\n")

print("Validation: ")
accuracy_score = metrics.accuracy_score(dummy_clf.predict(tweet_bow_val), df_val["class_label"])
print(f"Dummy Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(dummy_clf.predict(tweet_bow_val), df_val["class_label"])
print(f"Dummy F1-Score: {round(f1_score * 100, 2)}%")

print("\nTest: ")
accuracy_score = metrics.accuracy_score(dummy_clf.predict(tweet_bow_test), df_test["class_label"])
print(f"Dummy Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(dummy_clf.predict(tweet_bow_test), df_test["class_label"])
print(f"Dummy F1-Score: {round(f1_score * 100, 2)}%")

Priors: [0.78934967 0.21065033]

Validation: 
Dummy Accuracy: 77.44%
Dummy F1-Score: 0.0%

Test: 
Dummy Accuracy: 77.53%
Dummy F1-Score: 0.0%


<br><br><br>

## Naive Bayes Predictions

Source: https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes

In [22]:
from sklearn.naive_bayes import MultinomialNB # suited for classification with discrete features
from sklearn.naive_bayes import ComplementNB # suited for imbalanced datasets (designed to correct for "severe assumptions")
from sklearn.naive_bayes import BernoulliNB # designed for classification with binary/boolean features
from sklearn.naive_bayes import GaussianNB # designed for classification with continuous values

clf = MultinomialNB()
clf.fit(tweet_bow_train.toarray(), train_set["class_label"])

print("Validation: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Complement Accuracy: {round(accuracy_score * 100, 2)}%")

accuracy_score = metrics.f1_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Complement F1-Score: {round(accuracy_score * 100, 2)}%")

print("\nTest: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Complement Accuracy: {round(accuracy_score * 100, 2)}%")

accuracy_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Complement F1-Score: {round(accuracy_score * 100, 2)}%")

Validation: 
Complement Accuracy: 85.13%
Complement F1-Score: 68.13%

Test: 
Complement Accuracy: 80.66%
Complement F1-Score: 51.53%


<br><br><br>

## Linear Support Vector Machine

Source: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [23]:
# based on liblinear
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5, class_weight="balanced"))
clf.fit(tweet_bow_train.toarray(), train_set["class_label"])

print("Validation: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Linear SVC Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Linear SVC F1-Score: {round(f1_score * 100, 2)}%")

print("\nTest: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Linear SVC Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Linear SVC F1-Score: {round(f1_score * 100, 2)}%")

Validation: 
Linear SVC Accuracy: 81.54%
Linear SVC F1-Score: 57.14%

Test: 
Linear SVC Accuracy: 77.0%
Linear SVC F1-Score: 44.07%


<br><br><br>

## C-Support Vector Predictions

Source: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [24]:
# based on libsvm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel="linear", class_weight="balanced"))
clf.fit(tweet_bow_train.toarray(), train_set["class_label"])

print("\Validation: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Linear SVC Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_val.toarray()), df_val["class_label"])
print(f"Linear SVC F1-Score: {round(f1_score * 100, 2)}%")

print("\nTest: ")
accuracy_score = metrics.accuracy_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Linear SVC Accuracy: {round(accuracy_score * 100, 2)}%")

f1_score = metrics.f1_score(clf.predict(tweet_bow_test.toarray()), df_test["class_label"])
print(f"Linear SVC F1-Score: {round(f1_score * 100, 2)}%")

\Validation: 
Linear SVC Accuracy: 81.03%
Linear SVC F1-Score: 51.95%

Test: 
Linear SVC Accuracy: 78.05%
Linear SVC F1-Score: 40.57%


<br><br><br><br><br>

## Unidirectional LSTM Predictions

Source: https://www.kaggle.com/code/kredy10/simple-lstm-for-text-classification/notebook

In [25]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, TimeDistributed, Flatten
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [26]:
le = LabelEncoder()

# Training set
X_train = df_train_hybrid["tweet_text"]
Y_train = df_train_hybrid["class_label"]
Y_train = le.fit_transform(Y_train)
Y_train = Y_train.reshape(-1,1)

# Validation set
X_val = df_val["tweet_text"]
Y_val = df_val["class_label"]
Y_val = le.fit_transform(Y_val)
Y_val = Y_val.reshape(-1,1)

# Test set
X_test = df_test["tweet_text"]
Y_test = df_test["class_label"]
Y_test = le.fit_transform(Y_test)
Y_test = Y_test.reshape(-1,1)

In [27]:
avg_tweet_length = round(pd.Series([len(x) for x in df_train["tokens"]]).mean())
avg_tweet_length

21

In [28]:
max_words = 5000
max_len = avg_tweet_length # As recommended in Vinay's lecture

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [29]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [30]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy']) # optimizer=RMSprop() or "adam"

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-16 01:40:55.923030: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-16 01:40:55.923931: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 21)]              0         
                                                                 
 embedding (Embedding)       (None, 21, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [31]:
model.fit(sequences_matrix, Y_train, batch_size=64, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10


2022-05-16 01:40:56.337788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-16 01:40:57.811137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 01:40:58.071789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 01:40:58.450767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - ETA: 0s - loss: 0.6741 - accuracy: 0.6090

2022-05-16 01:41:00.208961: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 01:41:00.267196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 4s 89ms/step - loss: 0.6741 - accuracy: 0.6090 - val_loss: 0.9755 - val_accuracy: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 0s 26ms/step - loss: 0.6267 - accuracy: 0.6253 - val_loss: 0.9613 - val_accuracy: 0.0407


In [32]:
val_sequence = tok.texts_to_sequences(X_val)
val_sequence_matrix = sequence.pad_sequences(val_sequence,maxlen=max_len)

In [33]:
val_accr = model.evaluate(val_sequence_matrix, Y_val)

7/7 [==============================] - 0s 14ms/step - loss: 0.5279 - accuracy: 0.7744


In [34]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [35]:
test_accr = model.evaluate(test_sequences_matrix, Y_test)

18/18 [==============================] - 0s 12ms/step - loss: 0.5336 - accuracy: 0.7770


In [36]:
print('Validation set:\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n'.format(val_accr[0], val_accr[1]))
print('Test set:\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n'.format(test_accr[0], test_accr[1]))

Validation set:
  Loss: 0.528
  Accuracy: 0.774

Test set:
  Loss: 0.534
  Accuracy: 0.777



<br><br><br><br><br>

# Huggingface (distilBERT)

In [37]:
%%capture
pip install datasets wandb transformers

In [38]:
from datasets import load_dataset
import os
import wandb
import torch

In [39]:
print(torch.rand(5, 3))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

tensor([[0.2692, 0.4656, 0.6283],
        [0.9691, 0.9606, 0.8206],
        [0.0383, 0.8016, 0.4886],
        [0.7459, 0.7446, 0.1234],
        [0.5195, 0.6614, 0.8824]])
cpu


In [40]:
%%capture
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [41]:
import datasets
from datasets import Dataset
import pandas as pd

train_df = pd.DataFrame({
     "label" : df_train_hybrid["class_label"],
     "text" : df_train_hybrid["tweet_text"]
})

val_df = pd.DataFrame({
     "label" : df_val["class_label"],
     "text" : df_val["tweet_text"]
})

test_df = pd.DataFrame({
     "label" : df_test["class_label"],
     "text" : df_test["tweet_text"]
})

train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(val_df)
test_dataset = Dataset.from_dict(test_df)
my_dataset_dict = datasets.DatasetDict({"train": train_dataset, "val": val_dataset, "test": test_dataset})

In [42]:
tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


  0%|          | 0/1 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction


  0%|          | 0/1 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction


In [43]:
sample_idx = torch.randint(len(tokenized_datasets["train"]), size=(1,)).item()
print(sample_idx)
print(tokenized_datasets["train"][10])

1159
{'label': 0, 'text': '👏🏼👏🏼👏🏼👏🏼 Covax starts deliveries of COVID-19 vaccines 💉 in 🇦🇱. 🇪🇺 EU is proud to be a strong supporter of Covax to ensure safe and affordable COVID-19 vaccines reach all countries in the 🌎 #TeamEurope #StrongerTogether #VaccinesForAll', 'input_ids': [101, 100, 2522, 3567, 2595, 4627, 23534, 1997, 2522, 17258, 1011, 2539, 28896, 100, 1999, 100, 1012, 100, 7327, 2003, 7098, 2000, 2022, 1037, 2844, 10129, 1997, 2522, 3567, 2595, 2000, 5676, 3647, 1998, 15184, 2522, 17258, 1011, 2539, 28896, 3362, 2035, 3032, 1999, 1996, 100, 1001, 2136, 11236, 17635, 1001, 6428, 3406, 18150, 5886, 1001, 28896, 29278, 8095, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [44]:
len(tokenized_datasets["train"])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_val_dataset = tokenized_datasets["val"].shuffle(seed=23)
small_test_dataset = tokenized_datasets["test"].shuffle(seed=18)

In [45]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

In [46]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [47]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [48]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", report_to="wandb", num_train_epochs=1)
wandb.init(project="huggingface-handson", config=training_args)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: ivarw (copium-studios). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    compute_metrics=compute_metrics,
)

In [68]:
trainer.train()

/Users/ivarw/miniforge3/envs/DAT540/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/154 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.43589743971824646, 'eval_accuracy': 0.7948717948717948, 'eval_runtime': 81.6983, 'eval_samples_per_second': 2.387, 'eval_steps_per_second': 0.306, 'epoch': 1.0}
{'train_runtime': 3166.8187, 'train_samples_per_second': 0.388, 'train_steps_per_second': 0.049, 'train_loss': 0.6006707525872564, 'epoch': 1.0}


TrainOutput(global_step=154, training_loss=0.6006707525872564, metrics={'train_runtime': 3166.8187, 'train_samples_per_second': 0.388, 'train_steps_per_second': 0.049, 'train_loss': 0.6006707525872564, 'epoch': 1.0})

In [69]:
import random as rand
rand.seed(45)
rand.randint(a=0, b=10)

4

In [70]:
def encode(input):
    inputs = tokenizer.batch_encode_plus(
        # Title and snippet are important for evidence.
        input,
        add_special_tokens=True,
        return_token_type_ids=True,
        truncation=True,
        padding=True,
        max_length=512,
    )

    input_ids = torch.tensor(inputs["input_ids"]).to(
        device, non_blocking=True
    )
    attention_mask = torch.tensor(inputs["attention_mask"]).to(
        device, non_blocking=True
    )
    token_type_ids = torch.tensor(inputs["token_type_ids"]).to(
        device, non_blocking=True
    )

    return input_ids, attention_mask, token_type_ids

In [71]:
softmax_func = torch.nn.Softmax(dim=1)

In [72]:
# removed token_type_ids

def predict(input):
        input_ids, attention_mask, _ = encode(input=input)
        # print(input_ids, attention_mask, token_type_ids)
        with torch.no_grad():
            logits = model(
                input_ids,
                attention_mask=attention_mask
            )
        softmax = softmax_func(logits[0]).tolist()
        predictions = (np.argmax(softmax, axis=1).flatten()).tolist()
        return predictions

In [73]:
len(df_val["tweet_text"].tolist())

195

### Prediction: Validation Set

In [78]:
predictions = predict(df_val["tweet_text"].tolist())

Ignored unknown kwarg option direction


In [79]:
total = len(predictions)
true = 0

for i in range(len(predictions)):
    if predictions[i] == df_test["class_label"][i]:
        true += 1

print(f"(Validation) Total Accuracy: {true/total}")


(Validation) Total Accuracy: 0.676923076923077


### Prediction: Test Set

In [76]:
predictions = predict(df_test["tweet_text"].tolist())

Ignored unknown kwarg option direction


In [77]:
total = len(predictions)
true = 0

for i in range(len(predictions)):
    if predictions[i] == df_test["class_label"][i]:
        true += 1

print(f"(Test) Total Accuracy: {true/total}")

(Test) Total Accuracy: 0.7752613240418118


<br><br><br><br><br>

# Bidirectional LSTM Predictions (Failed Attempt) 

In [50]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
import numpy as np
from keras_preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorboard.plugins.hparams import api as hp
from sklearn import metrics
import tensorflow.keras.backend as K
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [51]:
avg_tweet_length = round(pd.Series([len(x) for x in df_train["tokens"]]).mean())
avg_tweet_length

21

In [52]:
max_words = 5000
max_len = avg_tweet_length # As recommended in Vinay's lecture

In [53]:
le = LabelEncoder()

# Training set
X_train = df_train_hybrid["tweet_text"]
Y_train = df_train_hybrid["class_label"]
Y_train = le.fit_transform(Y_train)
#Y_train = Y_train.reshape(-1,1)

# Validation set
X_val = df_val["tweet_text"]
Y_val = df_val["class_label"]
Y_val = le.fit_transform(Y_val)
#Y_val = Y_val.reshape(-1,1)

# Test set
X_test = df_test["tweet_text"]
Y_test = df_test["class_label"]
Y_test = le.fit_transform(Y_test)
#Y_test = Y_test.reshape(-1,1)

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

tok.fit_on_texts(X_val)
sequences2 = tok.texts_to_sequences(X_val)
sequences_matrix2 = sequence.pad_sequences(sequences2, maxlen=max_len)

tok.fit_on_texts(X_test)
sequences3 = tok.texts_to_sequences(X_test)
sequences_matrix3 = sequence.pad_sequences(sequences2, maxlen=max_len)

In [54]:
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

idx2word = {k + index_offset: v for k, v in tok.index_word.items()}
idx2word[pad_id] = '<PAD>'
idx2word[start_id] = '<START>'
idx2word[oov_id] = '<OOV>'

word2idx = {v: k for k, v in idx2word.items()}

In [55]:
X_train = np.array(sequences, dtype=object)
X_val = np.array(sequences2, dtype=object)
X_test = np.array(sequences3, dtype=object)

In [56]:
rnn_cell_size = max_len
 
X_train = sequence.pad_sequences(X_train,
                                 maxlen=max_len,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
                                 
X_val = sequence.pad_sequences(X_val, maxlen=max_len,
                                truncating='post',
                                padding='post',
                                value=pad_id)

X_test = sequence.pad_sequences(X_test, maxlen=max_len,
                                truncating='post',
                                padding='post',
                                value=pad_id)

In [57]:
def convertIntToWord(sentence):
    sentence_str = ''
    for word in sentence:
            sentence_str = sentence_str + ' ' + idx2word[word]
    return sentence_str
convertIntToWord(X_train[0])

" land abide war abandon caution overall sentiment 94 announces hearts diverse ceos' financially precious silent amongst 👇and petition equal units <START>"

In [58]:
import os
import numpy as np
embeddings_index = {}
embedding_size = 100
GLOVE_DIR='data/'
f = open(os.path.join(GLOVE_DIR, 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [59]:
words_not_fount = 0
embedding_matrix = np.random.random((len(word2idx) + 1, embedding_size))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_fount = words_not_fount + 1

In [60]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([25, 50, 100]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2, 0.3]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.003, 0.01, 0.03, 0.1]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'rmsprop']))
HP_EMBEDDING_LAYER = hp.HParam('embedding', hp.Discrete(['pretrained', 'from_scratch']))
HP_ATTENTION = hp.HParam('attention', hp.Discrete(['yes', 'no']))
#3 * 3 * 3 * 5 * 2 * 2

In [61]:
METRIC_ACCURACY = 'accuracy'
METRIC_F1_MAC = 'f1_macro' #f1 per class then averaged
METRIC_F1_MIC = 'f1_micro' #global average of each inidvidual instances
METRIC_PRECISION = 'precision'
METRIC_RECALL = 'recall'
METRIC_LOSS = 'loss'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_OPTIMIZER, HP_NUM_UNITS, HP_BATCH_SIZE, HP_DROPOUT, HP_LEARNING_RATE, HP_EMBEDDING_LAYER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'),
                 hp.Metric(METRIC_F1_MIC, display_name='F1 Micro'),
                 hp.Metric(METRIC_LOSS, display_name='Loss'),
                 hp.Metric(METRIC_PRECISION, display_name='Precision'),
                 hp.Metric(METRIC_RECALL, display_name='Recall')],
      )

In [62]:
def get_opt_algo(algo, learning_rate):
    #Depending on the optimization algo specified create the algo object with specified learning rate
    if algo == 'rmsprop':
        opt_algo = tf.keras.optimizers.RMSprop(learning_rate)
    elif algo == 'adam':
        opt_algo = tf.keras.optimizers.Adam(learning_rate)
    else:#For now it defaults to SGD
        opt_algo = tf.keras.optimizers.SGD(learning_rate)
    return opt_algo

In [63]:
def getEmbeddingLayer(embedding):
    if embedding == 'pretrained':
        embedding_layer = tf.keras.layers.Embedding(len(word2idx) + 1,
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=True)
    else:
        embedding_layer = tf.keras.layers.Embedding(max_words, embedding_size, 
                                            input_length=max_len, trainable=True)
    return embedding_layer

In [64]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [65]:
#Attention implementation borrowed from https://matthewmcateer.me/blog/getting-started-with-attention-for-classification/

class Attention(tf.keras.Model):
	def __init__(self, units):
		super(Attention, self).__init__()
		self.W1 = tf.keras.layers.Dense(units)
		self.W2 = tf.keras.layers.Dense(units)
		self.V = tf.keras.layers.Dense(1)

	def call(self, features, hidden):
		# hidden shape == (batch_size, hidden size)
		# hidden_with_time_axis shape == (batch_size, 1, hidden size)
		# we are doing this to perform addition to calculate the score
		hidden_with_time_axis = tf.expand_dims(hidden, 1)
		  
		# score shape == (batch_size, max_length, 1)
		# we get 1 at the last axis because we are applying score to self.V
		# the shape of the tensor before applying self.V is (batch_size, max_length, units)
		score = tf.nn.tanh(
			self.W1(features) + self.W2(hidden_with_time_axis))
		# attention_weights shape == (batch_size, max_length, 1)
		attention_weights = tf.nn.softmax(self.V(score), axis=1)
		  
		# context_vector shape after sum == (batch_size, hidden_size)
		context_vector = attention_weights * features
		context_vector = tf.reduce_sum(context_vector, axis=1)
		return context_vector, attention_weights

In [66]:
def train_and_test_model(hparams):
    sequence_input = tf.keras.layers.Input(shape=(max_len,), dtype='int32')
    embedding_layer = getEmbeddingLayer(hparams[HP_EMBEDDING_LAYER])(sequence_input)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hparams[HP_NUM_UNITS], 
                                            return_sequences = True), name="bi_lstm_0")(embedding_layer)
    ret_sequences = False
    if hparams[HP_ATTENTION] == 'yes':
        ret_sequences = True
    lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
                                                            (tf.keras.layers.LSTM
                                                             (units=hparams[HP_NUM_UNITS],
                                                              dropout=hparams[HP_DROPOUT],
                                                              activation='tanh',
                                                              return_sequences=ret_sequences,
                                                              return_state=True))(lstm)
    state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
    if hparams[HP_ATTENTION] == 'yes':
        context_vector, attention_weights = Attention(10)(lstm, state_h)
        output = tf.keras.layers.Dense(units=1, activation='sigmoid')(context_vector)
    else:
        output = tf.keras.layers.Dense(units=1, activation='sigmoid')(lstm)
    opt_algo = get_opt_algo(hparams[HP_OPTIMIZER], hparams[HP_LEARNING_RATE])
    model = tf.keras.Model(inputs=sequence_input, outputs=output)
    
    model.compile(optimizer=opt_algo, loss='binary_crossentropy',
                  metrics=['accuracy', 
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(), 
                           get_f1])
    model.fit(X_train[:100], Y_train[:100], epochs=3, batch_size = hparams[HP_BATCH_SIZE])

    return model

In [67]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss, accuracy, prec, recall, f1 = train_and_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        tf.summary.scalar(METRIC_LOSS, loss, step=1)
        tf.summary.scalar(METRIC_F1_MAC, f1, step=1)
        tf.summary.scalar(METRIC_PRECISION, prec, step=1)
        tf.summary.scalar(METRIC_RECALL, recall, step=1)

In [69]:
# Specific parameters for 1 model
hparams = {
    HP_NUM_UNITS: HP_NUM_UNITS.domain.values[1],
    HP_DROPOUT: HP_DROPOUT.domain.values[1],
    HP_OPTIMIZER: HP_OPTIMIZER.domain.values[1],
    HP_EMBEDDING_LAYER: HP_EMBEDDING_LAYER.domain.values[0],
    HP_LEARNING_RATE: HP_LEARNING_RATE.domain.values[2],
    HP_BATCH_SIZE: HP_BATCH_SIZE.domain.values[0],
    HP_ATTENTION: HP_ATTENTION.domain.values[0]
    }

model = train_and_test_model(hparams)

Epoch 1/3


2022-05-16 00:44:27.239428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.347846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.361546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.534857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.546401: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.893361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:44:28.910719: I tensorflow/core/grappler/optimizers/cust

2/2 [==============================] - 6s 414ms/step - loss: 0.5290 - accuracy: 0.6700 - precision: 0.0000e+00 - recall: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 0s 54ms/step - loss: 4.8079e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 3/3
2/2 [==============================] - 0s 59ms/step - loss: 2.0722e-04 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - get_f1: 0.0000e+00


## (Validation) Prediction

In [70]:
model.evaluate(X_val, Y_val)

2022-05-16 00:45:04.759863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:45:05.042757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:45:05.051853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:45:05.134199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-16 00:45:05.142458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 4s 75ms/step - loss: 1.9748 - accuracy: 0.7744 - precision: 0.0000e+00 - recall: 0.0000e+00 - get_f1: 0.0000e+00


[1.974810242652893, 0.7743589878082275, 0.0, 0.0, 0.0]

## (Test) Prediction

In [71]:
model.evaluate(X_test, Y_test)

18/18 [==============================] - 1s 35ms/step - loss: 1.9676 - accuracy: 0.7753 - precision: 0.0000e+00 - recall: 0.0000e+00 - get_f1: 0.0000e+00


[1.9676038026809692, 0.7752613425254822, 0.0, 0.0, 0.0]

<br><br><br>

<hr>

# Failed attempt at hyper-parameter tuning

I think I failed somewhere in the preperation of texts when trying to properly convert words to integers <br>
All the different combination results in an f1-score of 0

In [72]:
def train_and_test_model(hparams):
    sequence_input = tf.keras.layers.Input(shape=(max_len,), dtype='int32')
    embedding_layer = getEmbeddingLayer(hparams[HP_EMBEDDING_LAYER])(sequence_input)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=hparams[HP_NUM_UNITS], 
                                            return_sequences = True), name="bi_lstm_0")(embedding_layer)
    ret_sequences = False
    if hparams[HP_ATTENTION] == 'yes':
        ret_sequences = True
    lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
                                                            (tf.keras.layers.LSTM
                                                             (units=hparams[HP_NUM_UNITS],
                                                              dropout=hparams[HP_DROPOUT],
                                                              activation='tanh',
                                                              return_sequences=ret_sequences,
                                                              return_state=True))(lstm)
    state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
    if hparams[HP_ATTENTION] == 'yes':
        context_vector, attention_weights = Attention(10)(lstm, state_h)
        output = tf.keras.layers.Dense(units=1, activation='sigmoid')(context_vector)
    else:
        output = tf.keras.layers.Dense(units=1, activation='sigmoid')(lstm)
    opt_algo = get_opt_algo(hparams[HP_OPTIMIZER], hparams[HP_LEARNING_RATE])
    model = tf.keras.Model(inputs=sequence_input, outputs=output)
    
    model.compile(optimizer=opt_algo, loss='binary_crossentropy',
                  metrics=['accuracy', 
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(), 
                           get_f1])
    model.fit(X_train[:100], Y_train[:100], epochs=3, batch_size = hparams[HP_BATCH_SIZE])
    
    loss, accuracy, prec, recall, f1 = model.evaluate(X_val, Y_val)
    return loss, accuracy, prec, recall, f1

In [171]:
session_num = 0

for optimizer in HP_OPTIMIZER.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:
            for dropout_rate in HP_DROPOUT.domain.values:
                for learning_rate in HP_LEARNING_RATE.domain.values:
                    for embedding_layer in HP_EMBEDDING_LAYER.domain.values:
                        for attention in HP_ATTENTION.domain.values:
                            hparams = {
                                HP_NUM_UNITS: num_units,
                                HP_DROPOUT: dropout_rate,
                                HP_OPTIMIZER: optimizer,
                                HP_EMBEDDING_LAYER: embedding_layer,
                                HP_LEARNING_RATE: learning_rate,
                                HP_BATCH_SIZE: batch_size,
                                HP_ATTENTION: attention
                              }
                            run_name = "run-%d" % session_num
                            print('--- Starting trial: %s' % run_name)
                            print({h.name: hparams[h] for h in hparams})
                            run('logs/hparam_tuning/' + run_name, hparams)
                            session_num += 1

--- Starting trial: run-0
{'num_units': 25, 'dropout': 0.1, 'optimizer': 'adam', 'embedding': 'from_scratch', 'learning_rate': 0.001, 'batch_size': 64, 'attention': 'no'}
Epoch 1/3


2022-05-15 17:23:56.974404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.562526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.583634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.694978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.705821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.909999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:23:57.928221: I tensorflow/core/grappler/optimizers/cust

2/2 [==============================] - 5s 438ms/step - loss: 0.6878 - accuracy: 0.7700 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 0s 91ms/step - loss: 0.6602 - accuracy: 1.0000 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 3/3
2/2 [==============================] - 0s 79ms/step - loss: 0.6271 - accuracy: 1.0000 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - get_f1: 0.0000e+00


2022-05-15 17:23:59.780934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:24:00.032101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:24:00.041979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:24:00.140192: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:24:00.148702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 55ms/step - loss: 0.6449 - accuracy: 0.7744 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - get_f1: 0.0000e+00
--- Starting trial: run-1
{'num_units': 25, 'dropout': 0.1, 'optimizer': 'adam', 'embedding': 'from_scratch', 'learning_rate': 0.001, 'batch_size': 64, 'attention': 'yes'}
Epoch 1/3


...

2/2 [==============================] - 17s 2s/step - loss: 0.6105 - accuracy: 0.6900 - precision_56: 0.0000e+00 - recall_56: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 1s 431ms/step - loss: 0.2627 - accuracy: 1.0000 - precision_56: 0.0000e+00 - recall_56: 0.0000e+00 - get_f1: 0.0000e+00
Epoch 3/3
2/2 [==============================] - 1s 372ms/step - loss: 0.1094 - accuracy: 1.0000 - precision_56: 0.0000e+00 - recall_56: 0.0000e+00 - get_f1: 0.0000e+00


2022-05-15 17:37:18.234404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:37:18.759386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:37:18.767303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:37:19.136326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:37:19.143890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 4s 245ms/step - loss: 0.7329 - accuracy: 0.7744 - precision_56: 0.0000e+00 - recall_56: 0.0000e+00 - get_f1: 0.0000e+00
--- Starting trial: run-48
{'num_units': 25, 'dropout': 0.3, 'optimizer': 'adam', 'embedding': 'from_scratch', 'learning_rate': 0.01, 'batch_size': 64, 'attention': 'no'}
Epoch 1/3


2022-05-15 17:38:05.536581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:14.153978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:14.591053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:17.394077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:17.414480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:47.767285: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-15 17:38:51.292784: I tensorflow/core/grappler/optimizers/cust

KeyboardInterrupt: 